# Processing dataset from csv to parquet

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

In [2]:
# read S3 file into a data frame and show its data & metadata
df = pd.read_csv('raw_dataset.csv')
df.head(30)

,text
0,### Human: Who are you? ### Assistant: I am Ja...
1,### Human: Introduce yourself. ### Assistant: ...
2,### Human: What is your purpose ### Assistant:...
3,### Human: What are you? ### Assistant: I am a...
4,### Human: Introduce yourself. ### Assistant: ...
5,### Human: What can you do for me? ### Assista...
6,### Human: How were you created? ### Assistant...
7,### Human: Brief me about your features. ### A...
8,### Human: Start by introducing yourself. ### ...
9,### Human: What is your primary function? ### ...


In [29]:
# Function to process the text and extract the conversations
def process_conversation(text):
    conversations = []
    # print(text)
    # Split the text into human and assistant parts
    parts = text.split("###")
    for part in parts:
        part = part.strip()
        if part.startswith("Human:"):
            value = part.replace("Human:", "").strip()
            conversations.append({"from": "human", "value": value})
        elif part.startswith("Assistant:"):
            value = part.replace("Assistant:", "").strip()
            conversations.append({"from": "gpt", "value": value})
    
    return conversations
# Apply the function to each row in the dataframe
df['conversations'] = df['text'].apply(process_conversation)

# Keep only the 'conversations' column as requested
df_final = df[['conversations']]

# Display the processed dataframe to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Processed Conversations", dataframe=df_final)

In [26]:
df_final.head(30)

,conversations
0,"[{'from': 'human', 'value': 'Who are you?'}, {..."
1,"[{'from': 'human', 'value': 'Introduce yoursel..."
2,"[{'from': 'human', 'value': 'What is your purp..."
3,"[{'from': 'human', 'value': 'What are you?'}, ..."
4,"[{'from': 'human', 'value': 'Introduce yoursel..."
5,"[{'from': 'human', 'value': 'What can you do f..."
6,"[{'from': 'human', 'value': 'How were you crea..."
7,"[{'from': 'human', 'value': 'Brief me about yo..."
8,"[{'from': 'human', 'value': 'Start by introduc..."
9,"[{'from': 'human', 'value': 'What is your prim..."


In [27]:
from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi, HfFolder

# Convert the pandas dataframe to a Hugging Face dataset
hf_dataset = Dataset.from_pandas(df_final)

# Create a dataset dictionary
dataset_dict = DatasetDict({"train": hf_dataset})

# Push the dataset to Hugging Face
dataset_dict.push_to_hub("fotiecodes/jarvis-llama3.2-dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/fotiecodes/jarvis-llama3.2-dataset/commit/40c23d42bef9d865298b41be64f61592071417bd', commit_message='Upload dataset', commit_description='', oid='40c23d42bef9d865298b41be64f61592071417bd', pr_url=None, pr_revision=None, pr_num=None)